# Chapter 5: LangChain Introduction

These examples demonstrate various capabilities of LangChain.

Some of these capabilities did not work well with either small Ollama models (running on the CPU) or the Ollama Open AI compatible API (for example, the tools feature).

Therefore, I ran the examples in this file are run with the Anthropic Open AI compatible API, running `claude-3-5-haiku-20241022`

### Env File Setup

To provide environment variables to this Jupyter noteboko, the `dotenv` package is used. Create a file in the `chapter5/python` directory called `.env`, paste in these values, and upload them with your own. If you modify the file, just restart your Jupyter kernel and execute the first few cells again.

```env
LANGSMITH_TRACING=true
LANGSMITH_TRACING_V2=true
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_API_KEY="<your-api-key-here>"
LANGSMITH_PROJECT="<your-project-name-here>"
# If you don't have a MODEL_NAME, the llama3.2 model is used.
# You can comment them out to easily switch between them.
# MODEL_NAME="qwen2.5:3b"
# MODEL_NAME="gemma3:12b"
OPENAI_API_KEY="<your-api-key-here>"

# To use the Anthropic API, use this
#OPENAI_BASE_URL="https://api.anthropic.com/v1/"
#MODEL_NAME="claude-3-5-haiku-20241022"
```

In [3]:
import dotenv
dotenv.load_dotenv()

import os
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage 
from pydantic import validate_call

base_url = os.environ.get("OPENAI_BASE_URL") or "http://localhost:11434/v1"
api_key = os.environ.get("OPENAI_API_KEY") or "ollama"
model_name = os.environ.get("MODEL_NAME") or ("llama3.2" if api_key == "ollama" else "gpt-3.5-turbo")

### Create and test a langchain OpenAI client

In [2]:

def get_model():
    """Call this method to get a new model if you want one for some reason"""
    return ChatOpenAI(
        temperature=0.0,
        model_name=model_name,
        openai_api_key=api_key,
        openai_api_base=base_url)

chat_model = get_model()

display(chat_model.invoke("hi!"))

text = "I am looking to book a direct flight from New York to London departing on December 10th and returning on January 5th. Can you provide me with the available options, including airlines, flight times, and prices" 

messages = [
    SystemMessage(content="You're a helpful flight expert"),
    HumanMessage(content=text),
]

# display(chat_model.invoke(messages))


AIMessage(content='Hello! How are you doing today? Is there anything I can help you with?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 9, 'total_tokens': 29, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'claude-3-5-haiku-20241022', 'system_fingerprint': None, 'id': 'msg_014hFa7EgNE7QXenqw5YtKZX', 'finish_reason': 'stop', 'logprobs': None}, id='run-24b15899-13c2-4ccd-a4b0-ccac38f747d0-0', usage_metadata={'input_tokens': 9, 'output_tokens': 20, 'total_tokens': 29, 'input_token_details': {}, 'output_token_details': {}})

### Example of formatting a prompt template with interpolation

In [3]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Look at the following conversation {conversation} from the following service area {service_area} on {event_date_time} and return a sentiment"
)

conversation = """Customer: My new bike is missing a wheel! \
Chatbot: I'm sorry to hear that. Could I have your order number, please? \
Customer: It's #54321. \
Chatbot: Thank you. We'll send a replacement wheel today, and it'll arrive in two days. \
Customer: Make sure it does. This has been a hassle. \
Chatbot: Understandably so, and we apologize. You’ll also get a 20% discount on your next order for the inconvenience. \
Customer: Fine, thank you. \
Chatbot: You're welcome, and the confirmation is on its way. If there’s more I can do for you, just let me know."""

reminder_message = prompt_template.format(
    conversation=conversation,
    service_area="complaints",
    event_date_time="2023-10-19 14:30:00"
)
reminder_message

"Look at the following conversation Customer: My new bike is missing a wheel! Chatbot: I'm sorry to hear that. Could I have your order number, please? Customer: It's #54321. Chatbot: Thank you. We'll send a replacement wheel today, and it'll arrive in two days. Customer: Make sure it does. This has been a hassle. Chatbot: Understandably so, and we apologize. You’ll also get a 20% discount on your next order for the inconvenience. Customer: Fine, thank you. Chatbot: You're welcome, and the confirmation is on its way. If there’s more I can do for you, just let me know. from the following service area complaints on 2023-10-19 14:30:00 and return a sentiment"

In [5]:
from langchain.prompts import ChatPromptTemplate 

chat_template = ChatPromptTemplate.from_messages([ 
    ("system", "You are a health advisory bot for HealthHub Clinic. You can answer questions from the patient called {name}"), 
    ("ai", "Hi, {name} please ask me your question."), 
    ("human", "{user_input}"), 
])

messages = chat_template.format_messages(name="Lucy", user_input="What are the symptoms of the flu?")

display(messages)

chat_model.invoke(messages)

[SystemMessage(content='You are a health advisory bot for HealthHub Clinic. You can answer questions from the patient called Lucy', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi, Lucy please ask me your question.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What are the symptoms of the flu?', additional_kwargs={}, response_metadata={})]

AIMessage(content="Lucy, the flu (influenza) is a contagious respiratory illness caused by the influenza virus. The symptoms of the flu can vary from person to person, but common symptoms include:\n\n* Fever, usually high\n* Chills\n* Cough\n* Sore throat\n* Runny or stuffy nose\n* Headache\n* Fatigue (extreme tiredness)\n* Muscle or body aches\n* Diarrhea and vomiting (more common in children)\n\nIn some cases, the flu can also cause more severe symptoms, such as:\n\n* Pneumonia (infection of the lungs)\n* Bronchitis (inflammation of the airways)\n* Sinus and ear infections\n\nIf you're experiencing any of these symptoms, it's essential to consult with a healthcare professional for proper diagnosis and treatment.\n\nAt HealthHub Clinic, we can also provide you with guidance on how to manage your symptoms and prevent complications. Would you like me to recommend some over-the-counter medications or home remedies, Lucy?", additional_kwargs={'refusal': None}, response_metadata={'token_us

### ChatPromptTemplate example

Used to format a chat prompt (system, user and ai parts). This example creates a subclass to apply Pydantic validation to the template placeholders.

In [15]:
from langchain.prompts import ChatPromptTemplate
from pydantic import BaseModel, validator

class ProductivityChatPromptTemplate(ChatPromptTemplate, BaseModel):
    @validator("input_variables")
    def validate_input_variables(cls, v):
        required_vars = {"task", "time_available", "user_preferences"}
        if not required_vars.issubset(v):
            raise ValueError(f"Input variables must include: {required_vars}")
        return v

    def format_messages(self, **kwargs) -> str:
        messages = [
            ("system", "You are a virtual productivity assistant."),
            ("human", f"I need to {kwargs['task']} and I only have {kwargs['time_available']}."),
            ("human", f"My preference is to {kwargs['user_preferences']}."),
            ("ai", "Based on your task and preferences, here's my advice:")
            # The AI's response would be generated by the language model following this prompt.
        ]
        return self.construct_chat(messages)


/var/folders/m9/zg1dfhgn6r17gm0mlw6fg3yc0000gn/T/ipykernel_71239/1927113081.py:5: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("input_variables")


### StringPromptTemplate example

Used to format a basic string. This example creates a subclass to apply Pydantic validation to the template placeholders.

The example also shows how to use Python reflection to get function definitions and incorporate them into the chat, hypothetically for creating a tool to help explain your code.

In [11]:
import inspect


def get_source_code(function_name):
    # Get the source code of the function
    return inspect.getsource(function_name)

In [19]:
from langchain.prompts import StringPromptTemplate
from pydantic import BaseModel, validator

PROMPT = """\
Given the function name and source code, generate an English language explanation of the function.
Function Name: {function_name}
Source Code:
{source_code}
Explanation:
"""

class FunctionExplainerPromptTemplate(StringPromptTemplate, BaseModel):
    """A custom prompt template that takes in the function name as input, and formats the prompt template to provide the source code of the function."""

    @validator("input_variables")
    def validate_input_variables(cls, v):
        """Validate that the input variables are correct."""
        if len(v) != 1 or "function_name" not in v:
            raise ValueError("function_name must be the only input_variable.")
        return v

    def format(self, **kwargs) -> str:
        # Get the source code of the function
        source_code = get_source_code(kwargs["function_name"])

        # Generate the prompt to be sent to the language model
        prompt = PROMPT.format(
            function_name=kwargs["function_name"].__name__, source_code=source_code
        )
        return prompt

    def _prompt_type(self):
        return "function-explainer"
    
    
fn_explainer = FunctionExplainerPromptTemplate(input_variables=["function_name"])

# Generate a prompt for the function "get_source_code"
prompt = fn_explainer.format(function_name=get_source_code)
print(prompt)

print(chat_model.invoke(prompt).content)


/var/folders/m9/zg1dfhgn6r17gm0mlw6fg3yc0000gn/T/ipykernel_71239/1546726354.py:15: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("input_variables")


Given the function name and source code, generate an English language explanation of the function.
Function Name: get_source_code
Source Code:
def get_source_code(function_name):
    # Get the source code of the function
    return inspect.getsource(function_name)

Explanation:

**Function Explanation: `get_source_code`**

The `get_source_code` function is designed to retrieve the source code of a given Python function. It takes one argument, `function_name`, which is the name of the function for which you want to obtain its source code.

Here's how it works:

1. The function uses the built-in `inspect` module, which provides functions to help get information about live objects such as modules, classes, methods, etc.
2. Specifically, it calls the `getsource()` function from the `inspect` module, passing in the `function_name` argument.
3. The `getsource()` function returns the source code of the specified function as a string.

In essence, this function allows you to programmatically a

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.prompts import ChatPromptTemplate

intent_schema = ResponseSchema(name="intents",
                             description="Format the output as a single JSON object consisting of a key: intents, the intents key will be a list of objects with the following keys: intent, utterance, category")
response_schemas = [intent_schema]

intent_output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

intent_format_instructions = intent_output_parser.get_format_instructions()

print(intent_format_instructions)

intent_template = """
Create intents and utterances for a chatbot which will answer questions about a college, \
Create 4 examples of intents for the following categories: facilities and course_information. \
Ensure that each intent has 4 utterances total, including 2 long tail and 2 more common utterances.

{intent_format_instructions}

"""
prompt_template = ChatPromptTemplate.from_template(intent_template)
                                                           
messages = prompt_template.format_messages(intent_format_instructions=intent_format_instructions)
#messages = prompt_template.format_messages(intent_examples=intents)
response = chat_model.invoke(messages)
print("Got response")
# this will be type str
print(type(response.content))
display(response)
                               
output_dict = intent_output_parser.parse(response.content)
print(output_dict)
output_dict.get('intents')

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"intents": string  // Format the output as a single JSON object consisting of a key: intents, the intents key will be a list of objects with the following keys: intent, utterance, category
}
```
Got response
<class 'str'>


AIMessage(content='```json\n{\n    "intents": [\n        {\n            "intent": "facilities_library_info",\n            "category": "facilities",\n            "utterances": [\n                "What are the library\'s operating hours?",\n                "Can I access online resources from the library?",\n                "Tell me about the study spaces and resources available in the college library",\n                "Do you have information about the library\'s digital archives and research databases?"\n            ]\n        },\n        {\n            "intent": "facilities_campus_services",\n            "category": "facilities", \n            "utterances": [\n                "Where is the student health center located?",\n                "What counseling services are available on campus?",\n                "Can you provide details about disability support services and accommodations for students with special needs",\n                "Tell me about the campus wellness and mental healt

{'intents': [{'intent': 'facilities_library_info', 'category': 'facilities', 'utterances': ["What are the library's operating hours?", 'Can I access online resources from the library?', 'Tell me about the study spaces and resources available in the college library', "Do you have information about the library's digital archives and research databases?"]}, {'intent': 'facilities_campus_services', 'category': 'facilities', 'utterances': ['Where is the student health center located?', 'What counseling services are available on campus?', 'Can you provide details about disability support services and accommodations for students with special needs', 'Tell me about the campus wellness and mental health resources']}, {'intent': 'course_information_programs', 'category': 'course_information', 'utterances': ['What majors do you offer?', 'Can I see a list of undergraduate programs?', "I'm interested in learning about the interdisciplinary degree programs and unique academic tracks available at the

[{'intent': 'facilities_library_info',
  'category': 'facilities',
  'utterances': ["What are the library's operating hours?",
   'Can I access online resources from the library?',
   'Tell me about the study spaces and resources available in the college library',
   "Do you have information about the library's digital archives and research databases?"]},
 {'intent': 'facilities_campus_services',
  'category': 'facilities',
  'utterances': ['Where is the student health center located?',
   'What counseling services are available on campus?',
   'Can you provide details about disability support services and accommodations for students with special needs',
   'Tell me about the campus wellness and mental health resources']},
 {'intent': 'course_information_programs',
  'category': 'course_information',
  'utterances': ['What majors do you offer?',
   'Can I see a list of undergraduate programs?',
   "I'm interested in learning about the interdisciplinary degree programs and unique academ

### LCEL Example

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

prompt = ChatPromptTemplate.from_template("""
    Translate this text, without saying anything else: {text} to {language} 
"""
)
language = "French"
text = "what is the capital of england"

chain = prompt | chat_model | StrOutputParser()
chain.invoke({"text": text, "language": language})

"Quelle est la capitale de l'Angleterre ?"

In [7]:
#Filename: Sequential chain transcript processor 
import json
from langchain.chat_models import ChatOpenAI
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.schema.runnable import RunnableLambda

# Load transcripts data
with open('transcripts.json', 'r') as file:
    transcripts = json.load(file)

transcripts["conversations"] = transcripts["conversations"][:5]

# Define response schemas
response_schemas = [
    ResponseSchema(name="transactional_transcripts", description="Format the output as JSON list of conversations with the same JSON format as the input,add an category key to each conversation. Remember that JSON strings use\"", type="list"),
    ResponseSchema(name="faq", description="Format the output as JSON list of conversations with the same JSON format as the input, add an category key to each conversation. Remember that JSON strings use\"", type="list"),
]

# Define chat transcript template with placeholders for transcripts and format instructions
transcript_template = "Look at the following chat transcripts {transcripts} and categorize them into FAQ and transactional conversations in the following format {format_instructions}"


# Define prompt templates
transactional_categorization_prompt_template = HumanMessagePromptTemplate.from_template(transcript_template)

# Create output parser
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Get format instructions
format_instructions = output_parser.get_format_instructions()

# Create prompts
prompt = ChatPromptTemplate(
    messages=[transactional_categorization_prompt_template],
    input_variables=["transcripts"],
    partial_variables={"format_instructions": format_instructions},
)

# Define chain for transactional categorization
chain_one = prompt | chat_model | output_parser

#chain_one_result = chain_one.invoke({"transcripts": transcripts})

# Define intent response schemas
intent_response_schemas = [
    ResponseSchema(name="transactional_intents", description="Format the output as JSON list of conversation transcripts using the same format from the transactional_transcript list, add an intent key to each conversation", type="list"),
]

intent_transcript_template = "Look at the following chat transcripts {transactional_transcripts} Cluster these conversations by intent {intent_format_instructions}"


# Create intent prompt
intent_clustering_prompt_template = HumanMessagePromptTemplate.from_template(intent_transcript_template)

# Create intent output parser
intent_output_parser = StructuredOutputParser.from_response_schemas(intent_response_schemas)

# Get intent format instructions
intent_format_instructions = intent_output_parser.get_format_instructions()

# Create prompt for intent clustering
prompt_two = ChatPromptTemplate(
    messages=[intent_clustering_prompt_template],
    input_variables=["transactional_transcripts"],
    partial_variables={"intent_format_instructions": intent_format_instructions},
)

# Create chain for intent clustering
chain2 = (
    {"transactional_transcripts": chain_one}
    | prompt_two
    | chat_model
    | intent_output_parser
)

# Pass transcripts through chain inputs
chain_two_result = chain2.invoke({"transcripts": transcripts})
chain_two_result

{'transactional_intents': [{'category': 'maintenance_scheduling',
   'intent': 'service_booking',
   'live_chat_transcript': [{'role': 'customer',
     'message': 'Hello, I need to schedule a maintenance appointment for my ElectricZ Model 3. Can you help with that?'},
    {'role': 'support_agent',
     'message': "Certainly! Can you please provide your car's VIN and let me know what type of maintenance you require?"},
    {'role': 'customer',
     'message': 'My VIN is XYZ12345, and I need a routine checkup.'},
    {'role': 'support_agent',
     'message': "Thank you! I've scheduled your maintenance appointment. You'll receive a confirmation email shortly. Is there anything else I can assist you with?"}]},
  {'category': 'subscription_change',
   'intent': 'product_upgrade',
   'live_chat_transcript': [{'role': 'customer',
     'message': "Hello, I'd like to change my subscription plan from the ElectricA Model Y to the ElectricB Model Z. How can I do that?"},
    {'role': 'support_agen

In [10]:
chain_two_result.keys()

dict_keys(['transactional_intents'])

In [12]:
#filename: Parallel Chains in Langchain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableParallel
from langchain.schema import StrOutputParser

# prompt for querying news articles about a topic
news_prompt = ChatPromptTemplate.from_template("summarize recent news articles about {topic}")

# Chain for querying scientific papers about a topic
academic_prompt = ChatPromptTemplate.from_template("summarize recent scientific papers about {topic}")

# Chain for querying general web information about a topic
web_info_prompt = ChatPromptTemplate.from_template("provide a general overview of {topic} from web sources")


# Create a RunnableParallel instance with the three chains
# parallel_chain = RunnableParallel(news=news_chain, academic=academic_chain, web_info=web_info_chain)

parallel = RunnableParallel(
    news = news_prompt | chat_model,
    academic = academic_prompt | chat_model,
    web_info = web_info_prompt | chat_model
)

summarise_prompt = ChatPromptTemplate.from_template("""
summarize the following information from these different sources:

News source: {news}
Academic: {academic}
Web: {web_info}

Summary:
""")



# Invoke the parallel chain with a specific topic
#results = parallel_chain.invoke({"topic": "artificial intelligence"})

summarise_chain = parallel | summarise_prompt | chat_model

summarise_output = summarise_chain.invoke({"topic": "Economic policy"})

print(summarise_output.content)


Here's a comprehensive summary of economic policy based on the provided sources:

Economic Policy Overview:

Definition:
Economic policy consists of strategies and actions implemented by governments and central banks to influence a nation's economic conditions, growth, and financial well-being.

Key Objectives:
1. Economic growth
2. Price stability
3. Full employment
4. Sustainable development
5. Income distribution

Main Types of Economic Policy:

1. Fiscal Policy
- Government spending and taxation strategies
- Manages economic performance through budgets, tax rates, and public spending

2. Monetary Policy
- Controlled by central banks
- Manages money supply and interest rates
- Uses tools like interest rate adjustments and quantitative easing

3. Trade Policy
- Regulates international trade
- Focuses on import/export regulations and trade agreements

4. Industrial Policy
- Supports specific industries
- Provides subsidies and investment incentives

5. Structural Policy
- Addresses lo

### Demonstration of RunnableParallel

In [ ]:
from langchain.prompts import ChatPromptTemplate  
from langchain.schema.runnable import RunnableParallel, RunnableSequence

joke_prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
poem_prompt = ChatPromptTemplate.from_template("write a 2-line poem about {topic}")

parallel = RunnableParallel(
    joke = joke_prompt | chat_model,
    poem = poem_prompt | chat_model
)

summary_prompt = ChatPromptTemplate.from_template("""
Summarize the joke and poem about

Joke: {joke}
Poem: {poem}

Summary:
""")


sequence = parallel | summary_prompt | chat_model

output = sequence.invoke({"topic": "doctors"})
print(output)

content='Here\'s a summary of the joke and poem:\n\nJoke Summary:\nA patient tells his doctor that he thinks he\'s becoming a hypochondriac. The doctor humorously responds that the patient shouldn\'t worry, as being a hypochondriac is "nothing to be sick about" - a playful wordplay that pokes fun at the irony of worrying about being a worrier.\n\nPoem Summary:\nThe poem celebrates doctors as compassionate and knowledgeable professionals. It portrays them as healers with skilled hands, deep medical knowledge, and caring hearts. The poem emphasizes doctors as "guardians of life" who combine scientific expertise with hope, highlighting their critical role in healthcare and human well-being.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 157, 'prompt_tokens': 601, 'total_tokens': 758, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'claude-3-5-haiku-20241022', 'system_fingerprint': None, 'id': 'msg_016srCfhwpFkG

### Example of classifying input and routing

In [4]:
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch
from langchain.prompts import PromptTemplate
from typing import Literal
from langchain.output_parsers.openai_functions import PydanticAttrOutputFunctionsParser
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from operator import itemgetter
from langchain.schema.runnable import RunnablePassthrough
from pydantic import BaseModel

# Maintenance Department Template
maintenance_template = """You are an assistant at a car service center. \
You help customers book their cars for service. You will collect details such as the customer's name, \
car registration number, and their preferred date and time for the car collection.

Here is the service booking request:
{input}"""
maintenance_prompt = PromptTemplate.from_template(maintenance_template)

# Car Information Department Template
car_info_template = """You are knowledgeable about various car models and their features. \
You can provide detailed information about car specifications, models, and performance. \
If a question is outside your expertise, you recommend contacting the car manufacturer.

Here is the question:
{input}"""
car_info_prompt = PromptTemplate.from_template(car_info_template)

# Accounts Department Template
accounts_template = """You are well-versed in account management for car subscriptions. \
You can answer questions about billing, payment methods, and subscription plans. \
In cases of specific account issues, you advise contacting the accounts department directly.

Here is the question:
{input}"""
accounts_prompt = PromptTemplate.from_template(accounts_template)

# General Prompt Template for Other Queries
general_prompt = PromptTemplate.from_template(
    "You are a helpful assistant. Answer the FAQ question as accurately as you can.\n\n{input}"
)

# Branching Logic Based on Department
prompt_branch = RunnableBranch(
    (lambda x: x["topic"] == "maintenance", maintenance_prompt),
    (lambda x: x["topic"] == "car_info", car_info_prompt),
    (lambda x: x["topic"] == "accounts", accounts_prompt),
    general_prompt,
)


# Topic Classifier for Department Selection
class TopicClassifier(BaseModel):
    "Classify the topic of the user question"
    topic: Literal["maintenance", "car_info", "accounts", "general"]
    "The topic of the user question. One of 'maintenance', 'car_info', 'accounts', or 'general'."

classifier_function = convert_pydantic_to_openai_function(TopicClassifier)
llm = get_model().bind(
    functions=[classifier_function], function_call={"name": "TopicClassifier"}
)
parser = PydanticAttrOutputFunctionsParser(
    pydantic_schema=TopicClassifier, attr_name="topic"
)
classifier_chain = llm | parser

    
# Final Chain Assembly
final_chain = (
    RunnablePassthrough.assign(topic=itemgetter("input") | classifier_chain)
    | prompt_branch
    | get_model()
    | StrOutputParser()
)

# Example Invocation
# final_chain.invoke(
    #{"input": "How do I update my payment method for my car subscription?"}
    #{"input": "which is car with the longest range battery?"}
    # {"input": "whats the benefit of a subscription car"}
# )

/var/folders/m9/zg1dfhgn6r17gm0mlw6fg3yc0000gn/T/ipykernel_77851/1703045430.py:58: LangChainDeprecationWarning: The function `_convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  classifier_function = convert_pydantic_to_openai_function(TopicClassifier)


In [5]:
which_car_maintenance = final_chain.invoke(
    {"input": "Does a Toyota Corolla or a Honda Accord need maintenance more frequently?"}
)

OutputParserException: Could not parse function call: 'function_call'
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [21]:
print(which_car_maintenance)

Both Toyota Corolla and Honda Accord are known for their reliability, but they have slightly different maintenance schedules:

Toyota Corolla:
- Typically requires service every 5,000 miles or 6 months
- Regular maintenance includes oil changes, tire rotations, and inspections

Honda Accord:
- Usually needs service every 7,500 miles or 12 months
- Regular maintenance includes oil changes, tire rotations, and system checks

While the Accord may have slightly longer intervals between services, both cars benefit from following the manufacturer's recommended maintenance schedule. The exact frequency can depend on driving conditions, age of the vehicle, and individual usage.

Would you like to book a service appointment for your vehicle? I can help you schedule maintenance with our service center.
